# Funções LTN Relations Milenna

Funcoes iniciais

In [ ]:
!pip install ltn

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import numpy as np
import matplotlib.patches as patches
import math
import string
import torch
import ltn


In [ ]:
##parte do código milenna

def get_coordinates( bb ):
    x0 = bb[ 0 ]
    y0 = bb[ 1 ]
    x1 = bb[ 0 ] + bb[ -2 ]
    y1 = bb[ 1 ] + bb[ -1 ]
    return [ x0, y0, x1, y1 ]


def calculate_angle( bb1, bb2 ):
    c2 = np.array( (bb2[ 0 ], bb2[ 1 ]) )
    c1 = np.array( (bb1[ 0 ], bb1[ 1 ]) )

    x = c2 - c1

    return np.angle( x[ 0 ] + 1j * x[ 1 ], deg = True )




Retorna os elementos dos bbs

In [ ]:
def translate_bbs( bb1, bb2, is_class_first = False,
                   is_yolo_format = False ):
    """
    Translate bounding box coordinates based on specified conditions.
    Parameters:
        bb1 (tensor): The coordinates of the first bounding box.
        bb2 (tensor): The coordinates of the second bounding box.
        is_class_first (bool, optional): If True, assumes the class is the first element in each bounding box. Default is False.
        is_yolo_format (bool, optional): If True, assumes the bounding boxes are in YOLO format. Default is False.

    Returns:
        tuple: Translated coordinates of the first and second bounding boxes in the format (x, y, w, h).

    Example:
        bb1 = tf.constant([0.3, 0.3, 0.2, 0.2])
        bb2 = tf.constant([0.3, 0.3, 0.1, 0.1])
        translated_bb1, translated_bb2 = translate_bbs(bb1, bb2, is_class_first=True, is_yolo_format=True)
    """
    # if is_class_first:
    #     indices = (1, 2, 3, 4) if is_yolo_format else (0, 1, 2, 3)
    # else:
    #     indices = (0, 1, 2, 3) if is_yolo_format else (0, 1, 2, 3)
    # x1, y1, w1, h1 = bb1[ ..., indices ]
    # x2, y2, w2, h2 = bb2[ ..., indices ]
    # if is_yolo_format:
    #     x1, y1 = x1 - w1 / 2.0, y1 - h1 / 2.0
    #     x2, y2 = x2 - w2 / 2.0, y2 - h2 / 2.0
    if is_class_first :
        if is_yolo_format :
            x1, y1 = bb1[ ..., 1 ] - bb1[ ..., 3 ] / 2.0, bb1[ ..., 2 ] - bb1[ ..., 4 ] / 2.0
            x2, y2 = bb2[ ..., 1 ] - bb2[ ..., 3 ] / 2.0, bb2[ ..., 2 ] - bb2[ ..., 4 ] / 2.0
            w1, h1, w2, h2 = bb1[ ..., 3 ], bb1[ ..., 4 ], bb2[ ..., 3 ], bb2[ ..., 4 ]
        else :
            x1, y1, w1, h1 = bb1[ ..., 1 ], bb1[ ..., 2 ], bb1[ ..., 3 ], bb1[ ..., 4 ]
            x2, y2, w2, h2 = bb2[ ..., 1 ], bb2[ ..., 2 ], bb2[ ..., 3 ], bb2[ ..., 4 ]

    else :
        if is_yolo_format :
            x1, y1 = bb1[ ..., 0 ] - bb1[ ..., 2 ] / 2.0, bb1[ ..., 1 ] - bb1[ ..., 3 ] / 2.0
            x2, y2 = bb2[ ..., 0 ] - bb2[ ..., 2 ] / 2.0, bb2[ ..., 1 ] - bb2[ ..., 3 ] / 2.0
            w1, h1, w2, h2 = bb1[ ..., 2 ], bb1[ ..., 3 ], bb2[ ..., 3 ], bb2[ ..., 3 ]
        else :
            x1, y1, w1, h1 = bb1[ ..., 0 ], bb1[ ..., 1 ], bb1[ ..., 2 ], bb1[ ..., 3 ]
            x2, y2, w2, h2 = bb2[ ..., 0 ], bb2[ ..., 1 ], bb2[ ..., 2 ], bb2[ ..., 3 ]

    return (x1, y1, w1, h1), (x2, y2, w2, h2)



Funções que determinam duas bbs (overlap, partial e distinct)
Recebem em tensores e fazem a conversão dentro da função

--Overlap   | Verificar se pode voltar em bool (ajustar denovo)

In [ ]:

def O( bb1: tf.Tensor, bb2: tf.Tensor, is_yolo_format = True, is_class_first = False ):
    """
    Check if two bounding boxes overlap.
    Parameters:
        bb1 (tensor): The coordinates of the first bounding box.
        bb2 (tensor): The coordinates of the second bounding box.
        is_class_first (bool, optional): If True, assumes the class is the first element in each bounding box. Default is False.
        is_yolo_format (bool, optional): If True, assumes the bounding boxes are in YOLO format. Default is False.
    Returns:
        bool: True if the bounding boxes overlap, False otherwise.

    Example:
        bb1 = tf.constant( [ 0.3, 0.3, 0.2, 0.2 ] )
        bb2 = tf.constant( [ 0.3, 0.3, 0.1, 0.1 ] )

        print( 'Check for O (True): ', O( bbs = (bb1, bb2), is_yolo_format = True, is_class_first = False ) )
    """
    (x1, y1, w1, h1), (x2, y2, w2, h2) = translate_bbs( bb1, bb2, is_yolo_format = is_yolo_format, is_class_first = is_class_first )

    condition1 = tf.logical_and(tf.greater_equal( x2, x1 ), tf.less_equal( x2 + w2, x1 + w1 ))
    condition2 = tf.logical_and(tf.greater( y2, y1 ), tf.less( y2 + h2, y1 + h1 ))
    # print(tf.logical_or(condition1,condition2))
    overlap = tf.logical_or( condition1, condition2 )
    return overlap

PO

In [ ]:

# def PO(bbs, is_yolo_format=True):
def PO( bb1: tf.Tensor, bb2: tf.Tensor, is_yolo_format = True, is_class_first = False ):
    """
    Check if two bounding boxes partially overlap.

    Parameters:
        bb1 (tensor): The coordinates of the first bounding box.
        bb2 (tensor): The coordinates of the second bounding box.
        is_class_first (bool, optional): If True, assumes the class is the first element in each bounding box. Default is False.
        is_yolo_format (bool, optional): If True, assumes the bounding boxes are in YOLO format. Default is False.

    Returns:
        bool: True if the bounding boxes partially overlap, False otherwise.

    Note: (x1,y1) and (x2,y2) ordered pair ar located in lowest-left corner of the bounding box
    """
    (x1, y1, w1, h1), (x2, y2, w2, h2) = translate_bbs( bb1, bb2, is_yolo_format = is_yolo_format, is_class_first = is_class_first )
    '''
    mesmo que:
                # PO 1
            ((x2 < x1) and ((x2 + w2) > x1) and
             (((y2 < y1) and ((y2 + h2) > y1)) or ((y2 + h2) > (y1 + h1))))

            # PO 2
            or (((x2 < x1) and ((x2 + w2) > x1) and ((x2 + w2) < (x1 + w1))) and
                ((((y2 + h2) > (y1 + h1)) and (y2 < (y1 + h1))) or ((y2 < y1) and ((y2 + h2) > y1))))

            # PO 3
            or (((x2 > x1) and ((x2 + h2) < (x1 + h1))) and
                ((((y2 + h2) > (y1 + h1)) and (y2 < (y1 + h1))) or ((y2 < y1) and ((y2 + h2) > y1))))

            # PO 4
            or (((x2 > x1) and (x2 < (x1 + w1)) and ((x2 + w2) > (x1 + w1))) and
                ((((y2 + h2) < (y1 + h1)) and ((y2 + h2) > y1)) or ((y2 > y1) and (y2 < (y1 + h1)))))
    Essas condições são para elementos iteraveis (list), mas aqui é adaptado para uso em tf.Tensor()
    '''
    # Condition 1
    condition1 = tf.logical_and(
            tf.logical_and( x2 < x1, x2 + w2 > x1 ),
            tf.logical_or(
                    tf.logical_and( y2 < y1, y2 + h2 > y1 ),
                    y2 + h2 > y1 + h1
            )
    )
    # Condition 2
    condition2 = tf.logical_and(
            tf.logical_and( x2 < x1, x2 + w2 > x1 ),
            tf.logical_and(
                    x2 + w2 < x1 + w1,
                    tf.logical_or(
                            tf.logical_and( y2 + h2 > y1 + h1, y2 < y1 + h1 ),
                            tf.logical_and( y2 < y1, y2 + h2 > y1 )
                    )
            )
    )
    # Condition 3
    condition3 = tf.logical_and(
            tf.logical_and( x2 > x1, x2 + h2 < x1 + h1 ),
            tf.logical_or(
                    tf.logical_and( y2 + h2 > y1 + h1, y2 < y1 + h1 ),
                    tf.logical_and( y2 < y1, y2 + h2 > y1 )
            )
    )
    # Condition 4
    condition4 = tf.logical_and(
            tf.logical_and( x2 > x1, tf.logical_and( x2 < x1 + w1, x2 + w2 > x1 + w1 ) ),
            tf.logical_or(
                    tf.logical_and( y2 + h2 < y1 + h1, y2 + h2 > y1 ),
                    tf.logical_and( y2 > y1, tf.logical_and( y2 < y1 + h1, y2 < y1 + h1 ) )
            )
    )
    # Final condition, combining all the above conditions with logical OR
    final_condition = tf.logical_or(
            tf.logical_or( condition1, condition2 ),
            tf.logical_or( condition3, condition4 )
    )
    return final_condition


Disjoint

In [ ]:

def D( bb1: tf.Tensor, bb2: tf.Tensor, is_yolo_format = True, is_class_first = False ):
    """
    Check if two bounding boxes are disjoint.
    Parameters:
        bb1 (tuple): The coordinates of the first bounding box in the format (x1, y1, w1, h1).
        bb2 (tuple): The coordinates of the second bounding box in the format (x2, y2, w2, h2).
        is_class_first (bool, optional): If True, assumes the class is the first element in each bounding box. Default is False.
        is_yolo_format (bool, optional): If True, assumes the bounding boxes are in YOLO format. Default is False.
    Returns:
        bool: True if the bounding boxes are disjoint, False otherwise.
    """
    (x1, y1, w1, h1), (x2, y2, w2, h2) = translate_bbs( bb1, bb2, is_yolo_format = is_yolo_format, is_class_first = is_class_first )

    condition1 = tf.less( x2 + w2, x1 )
    condition2 = tf.greater( x2, x1 + w1 )
    condition3 = tf.greater( y2, y1 + h1 )
    condition4 = tf.less( y2 + h2, y1 )

    bool_tensor = tf.logical_or( tf.logical_or( condition1, condition2 ),
                                 tf.logical_or( condition3, condition4 ) )
    return bool_tensor


Verificar que as bbs para esses exemplo foram definidas como:


```
bb1 = tf.constant( [ 0.5, 0.4, 0.2, 0.3 ] )
bb2 = tf.constant( [ 0.5, 0.6, 0.2, 0.2 ] )
```



# Funções Spartial Relation

## Funções requisito pra gerar os Bbs

In [ ]:
def get_rectangle(bb, edcolor='r'):
    """
    Create a rectangle patch for a given bounding box.
    This function creates and returns a rectangle patch object using the specified bounding box coordinates. The rectangle
    is defined by the center point (bb[0], bb[1]), width bb[2], and height bb[3]. The edge color of the rectangle can be
    customized using the optional 'edcolor' parameter.

    Parameters:
    bb (tuple): The bounding box coordinates in the format (center_x, center_y, width, height).
    edcolor (str, optional): The edge color of the rectangle (default is 'r').

    Returns:
    patches.Rectangle: A rectangle patch object representing the bounding box.

    Rectangle(xy=(0, 0), width=100, height=100, angle=0)
    Note:
    - The function assumes the bounding box coordinates are in the form (center_x, center_y, width, height).
    - The default edge color is 'r' (red).
    - The face color of the rectangle is set to 'none', resulting in an empty rectangle.
    """
    return patches.Rectangle((bb[0] - bb[2] / 2, bb[1] - bb[3] / 2), bb[2], bb[3], linewidth=2, edgecolor=edcolor,
                             facecolor='none')


def get_corners_coord(bb):
    x0 = bb[0] - bb[-2] / 2
    y0 = bb[1] - bb[-1] / 2
    x1 = bb[0] + bb[-2] / 2
    y1 = bb[1] + bb[-1] / 2

    top_left = (x0, y0)
    top_right = (x1, y0)
    bottom_left = (x0, y1)
    bottom_right = (x1, y1)

    return [top_left, top_right, bottom_left, bottom_right]


def get_interested_angles(bb):
    """
    Get the list of interested angles for a given bounding box.

    This function calculates and returns a list of angles of interest for a given bounding box. The angles include
    0.0, 90.0, -90.0, and 180.0, representing horizontal, vertical, and diagonal orientations.

    Parameters:
    bb (tuple): The bounding box coordinates in the format (center_x, center_y, width, height).

    Returns:
    list: A list of angles representing the interested angles for the bounding box.

    Example:
    # >>> bounding_box = (50, 50, 100, 100)
    # >>> angles = get_interested_angles(bounding_box)
    # >>> print(angles)
    [0.0, 90.0, -90.0, 180.0, 45.0, -45.0, 135.0, -135.0]

    Note:
    - The function assumes the bounding box coordinates are in the form (center_x, center_y, width, height).
    - The additional diagonal angles are calculated based on the corners of the bounding box.
    """
    angles = [0.0, 90.0, -90.0, 180.0]
    corners = get_corners_coord(bb)

    boxes = [(coord[0], coord[1], 0.1 * bb[2], 0.1 * bb[3]) for coord in corners]
    diag_angles = [calculate_angle(box, bb) for box in boxes]

    # print('diag_angles : ', diag_angles)

    return angles + diag_angles

def calculate_angle(bb1, bb2):
  c2 = np.array((bb2[0], bb2[1]))
  c1 = np.array((bb1[0], bb1[1]))

  x = c2 - c1

  return np.angle(x[0] + 1j * x[1], deg=True)


Geradores auxiliares

In [ ]:
def  generate_equally_spaced_points(bounding_box):
    """
    Generate equally spaced points within a bounding box in clockwise manner.

    This function calculates and generates equally spaced points within a given bounding box. The points are distributed
    based on different angles of interest within the box.

    Parameters:
    bounding_box (tuple): The bounding box coordinates in the format (center_x, center_y, width, height).

    Returns:
    list: A list of (x, y) coordinates representing the equally spaced points within the bounding box.
    """
    center_x, center_y, width, height = bounding_box
    points = []

    # Calculate spacing between points
    spacing_horizontal = width / 8
    spacing_vertical = height / 8

    diagonal_length = math.sqrt(width ** 2 + height ** 2)
    spacing_diagonal = diagonal_length / 8

    angles = get_interested_angles(bounding_box)
    angles.sort()

    pos_angles = [x for x in angles if x >= 0.0]
    neg_angles = [x for x in angles if x < 0.0]

    # ajustar angulos positivos para ordem crescente
    pos_angles.sort()
    # ajustar angulos negativos para ordem crescente
    neg_angles.sort()

    angles = pos_angles + neg_angles
    # para cada bb das direções, contando apartir do centro (1, 2 , 3 bb)
    for i in range(1, 4):

        # percorrendo os angulos de forma horária (pois está organizado : 0* -> 90 -> 180* e 180* -> -90 -> 0*)

        for angle in angles:

            x_signal = 1.0 if angle > 90.0 or angle < -90.0 else -1.0
            y_signal = 1.0 if angle > 0.0 else -1.0

            # horizontal
            if angle in [0.0, 180.0]:
                x = center_x + (i * spacing_horizontal * x_signal)
                y = center_y

            # diagonal
            if angle not in [0.0, 90.0, -90.0, 180.0]:
                x = center_x - i * spacing_diagonal * math.cos(math.radians(angle))
                y = center_y - i * spacing_diagonal * math.sin(math.radians(angle))

            # vertical
            if angle in [90.0, -90.0]:
                x = center_x
                y = center_y - (i * spacing_vertical * y_signal)

            points.append((x, y))


    return points


In [ ]:
## gera os quadrados externos
def plot_comparisons_diagonals(bb, boxes):
    """
    Plot comparisons of bounding boxes in relation to a main bounding box.

    This function generates a matplotlib figure with four subplots to visualize the comparisons of bounding boxes with
    respect to a main bounding box. The center_box and the main bounding box are plotted on each subplot for comparison.

    Parameters:
    bb (tuple): The bounding box coordinates of the main box in the format (x, y, width, height).
    boxes (list): A list of bounding box coordinates in the format [(x1, y1, width1, height1), (x2, y2, width2, height2), ...].

    Returns:
    None

    Example:
    bb = (10, 20, 50, 30)
    boxes = [(15, 25, 40, 20), (20, 30, 35, 25), (25, 35, 30, 30)]
    plot_comparisons_diagonals(bb, boxes)
    """
    # fig, axes = plt.subplots(nrows=2, ncols=2)
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)

    # inSideLeftAltAbove, inSideLeftAltBelow, inSideRightAltAbove, inSideRightAltBelow
    name = list(string.ascii_uppercase)[:len(boxes)]

    # Plot center box and bb for comparision

    center_box = boxes[0]

    for ax in [ax1, ax2, ax3, ax4]:
        ax.add_patch(get_rectangle(bb, 'r'))
        ax.add_patch(get_rectangle(center_box, 'g'))
        ax.text(center_box[0], center_box[1], f'box : {name[0]}', ha='center', va='center', fontsize=12, color='blue')
        ax.invert_yaxis()
        ax.autoscale()

    # Add rectangles using patches.Rectangle()

    rect_inSideLeftAbove = get_rectangle(boxes[2], 'b')
    ax1.add_patch(rect_inSideLeftAbove)
    ax1.set_title(f'inSideLeftAltAbove : {inSideLeftAltAbove(boxes[2], center_box)}')
    ax1.text(boxes[2][0], boxes[2][1], f'box : {name[2]}', ha='center', va='center', fontsize=12, color='blue')

    rect_inSideRightAltAbove = get_rectangle(boxes[4], 'b')
    ax2.add_patch(rect_inSideRightAltAbove)
    ax2.set_title(f'inSideRightAltAbove : {inSideRightAltAbove(boxes[4], center_box)}')
    ax2.text(boxes[4][0], boxes[4][1], f'box : {name[4]}', ha='center', va='center', fontsize=12, color='blue')

    rect_inSideRightAltBelow = get_rectangle(boxes[6], 'b')
    ax3.add_patch(rect_inSideRightAltBelow)
    ax3.set_title(f'inSideRightAltBelow : {inSideRightAltBelow(boxes[6], center_box)}')
    ax3.text(boxes[6][0], boxes[6][1], f'box : {name[6]}', ha='center', va='center', fontsize=12, color='blue')

    rect_inSideLeftAltBelow = get_rectangle(boxes[8], 'b')
    ax4.add_patch(rect_inSideLeftAltBelow)
    ax4.set_title(f'inSideLeftAltBelow : {inSideLeftAltBelow(boxes[8], center_box)}')
    ax4.text(boxes[8][0], boxes[8][1], f'box : {name[8]}', ha='center', va='center', fontsize=15, color='blue')

    fig.tight_layout()
    plt.show()


**Pode ser nosso dataset**

In [ ]:
#Gera bounding boxes dentro de bounding boxes , no sentido anti-horário
def generate_eq_spaced_bbs_in_(bb, dictionary_with_letters=True):
    """
    Function to generate bounding boxes inside of a bounding box 'bb' in clockwise direction.

    Parameters:
    bb (list): list of bounding boxes in YOLO Format ([x_center, y_center, width, height]).
    dictionary_with_letters (bool): if boxes will be referred as Alphabetic letter ( A to Z ).

    Returns:
    bbs (list) : list of resulting bounding boxes

    The first element of the 'boxes' list is considered as the center_box. The subsequent bounding boxes are arranged
    in a clockwise manner starting from 0 degrees and progressing to 90 degrees, 180 degrees, -90 degrees, and the
    diagonal angle before 0 degrees.

    """

    # print(f'x : {bb[0]}, y : {bb[1]}')

    angles = get_interested_angles(bb)
    angles.sort()

    ## get boxes
    center_coord = generate_equally_spaced_points(bb)
    # print('center coordinates : ', center_coord)

    boxes = [[coord[0], coord[1], 0.1 * bb[2], 0.1 * bb[3]] for coord in center_coord]

    center_box = [bb[0], bb[1], 0.1 * bb[2], 0.1 * bb[3]]

    bbs = [center_box] + boxes


    if dictionary_with_letters:
        letters = list(string.ascii_uppercase)[:len(bbs) + 1]
        result_bbs = {letter: box for letter, box in zip(letters, bbs)}
        greater_box = [bb[0], bb[1], 2 * (result_bbs['N'][0] - bb[0]), 2 * (result_bbs['P'][1] - bb[1])]

        result_bbs[letters[-1]] = greater_box

        return result_bbs
    else:
        return bbs


In [ ]:
# plota as bbs com nomes
def plot_all_bb_with_names(bb, boxes, name=list(string.ascii_uppercase)):
    """
    Plot bounding boxes with corresponding names on a matplotlib figure.

    Parameters:
    bb (tuple): The bounding box coordinates of the main box (x, y, width, height).
    boxes (list): A list of bounding box coordinates (x, y, width, height).
    name (list, optional): A list of names corresponding to each box. Default is a list of uppercase letters.

    Returns:
    None

    Example:
    bb = (10, 20, 50, 30)
    boxes = [(15, 25, 40, 20), (20, 30, 35, 25), (25, 35, 30, 30)]
    plot_all_bb_with_names(bb, boxes)
    """
    name = list(string.ascii_uppercase)[:len(boxes)]
    print(f'len_boxes : {len(boxes)}, len_names : {len(name)},name : {name}')

    fig, ax = plt.subplots()
    ax.add_patch(get_rectangle(bb))
    ax.set_title('Boxes Generated')

    if isinstance(boxes, dict):
        boxes_keys = list(boxes.keys())

        for i in range(0, len(boxes)):

            key_at_index = boxes_keys[i]
            box = boxes[key_at_index]


            if i == 0:
                ed = 'orange'
            elif (i % 2 == 0 and not (i == '0') and not (i == len(boxes) - 1)):
                ed = 'red'
            elif i == len( boxes ) - 1 :
                ed = 'green'
            else:
                ed = 'magenta'

            ax.add_patch(get_rectangle(box, edcolor=ed))
            ax.text(box[0] + 0.3 * box[2], box[1] - 0.55 * box[3], f'box : {name[i]}', ha='center', va='center', fontsize=12,
                    color='blue')
    else:
      for i in range(0, len(boxes)):
        if i == 0:
            ed = 'orange'
        elif (i % 2 == 0 and not (i == '0') and not (i == len(boxes) - 1)):
            ed = 'red'
        elif i == len( boxes ) - 1 :
            ed = 'green'
        else:
            ed = 'magenta'

        ax.add_patch(get_rectangle(boxes[i], edcolor=ed))
        ax.text(boxes[i][0], boxes[i][1] - 0.10, f'box : {name[i]}', ha='center', va='center', fontsize=12,
                color='blue')



    ax.legend(handles=[patches.Patch(color='red', label='diagonals'),
                       patches.Patch(color='magenta', label='horizontal and vertical'),
                       patches.Patch(color='orange', label='center_box'),
                       patches.Patch(color='green', label='greater_box')])
    ax.invert_yaxis()
    fig.tight_layout()
    plt.show()


# # Podemos usar as funções que geram






In [ ]:
#
bb1 = [0.5, 0.5, 0.2, 0.6]
clockwise_bounding_boxes = generate_eq_spaced_bbs_in_(bb1)

In [ ]:
#Definindo um referencial para a primeira bounding box
bb1 = [0.5, 0.5, 0.2, 0.6]
clockwise_bounding_boxes = generate_eq_spaced_bbs_in_(bb1)

#Passando para tensores:
A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W, X, Y, Z = [
    (tf.constant((clockwise_bounding_boxes.get(letter))))
    for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
]

In [ ]:
clockwise_bounding_boxes

{'A': [0.5, 0.5, 0.020000000000000004, 0.06],
 'B': [0.475, 0.5, 0.020000000000000004, 0.06],
 'C': [0.475, 0.425, 0.020000000000000004, 0.06],
 'D': [0.5, 0.425, 0.020000000000000004, 0.06],
 'E': [0.525, 0.425, 0.020000000000000004, 0.06],
 'F': [0.525, 0.5, 0.020000000000000004, 0.06],
 'G': [0.525, 0.575, 0.020000000000000004, 0.06],
 'H': [0.5, 0.575, 0.020000000000000004, 0.06],
 'I': [0.475, 0.575, 0.020000000000000004, 0.06],
 'J': [0.45, 0.5, 0.020000000000000004, 0.06],
 'K': [0.45, 0.35, 0.020000000000000004, 0.06],
 'L': [0.5, 0.35, 0.020000000000000004, 0.06],
 'M': [0.5499999999999999, 0.35, 0.020000000000000004, 0.06],
 'N': [0.55, 0.5, 0.020000000000000004, 0.06],
 'O': [0.5499999999999999, 0.65, 0.020000000000000004, 0.06],
 'P': [0.5, 0.65, 0.020000000000000004, 0.06],
 'Q': [0.45, 0.65, 0.020000000000000004, 0.06],
 'R': [0.425, 0.5, 0.020000000000000004, 0.06],
 'S': [0.425, 0.275, 0.020000000000000004, 0.06],
 'T': [0.5, 0.275, 0.020000000000000004, 0.06],
 'U': [0

Ou utilizar a nossa própria:
  

In [ ]:
def generate_synthetic_bounding_boxes(num_boxes, img_size=100):
    """
    Generate synthetic bounding boxes for training.

    Parameters:
    num_boxes (int): Number of bounding boxes to generate.
    img_size (int): Size of the image.

    Returns:
    list: List of bounding boxes in the format (x_center, y_center, width, height).
    """
    boxes = []
    for _ in range(num_boxes):
        x_center = np.random.randint(10, img_size - 10)
        y_center = np.random.randint(10, img_size - 10)
        width = np.random.randint(10, img_size - x_center)
        height = np.random.randint(10, img_size - y_center)
        boxes.append((x_center, y_center, width, height))
    return boxes

In [ ]:
my_bouding_boxes = generate_synthetic_bounding_boxes(100);

# Inicio do trabalho


Ao trabalhar com tensores, vamos precisar de algumas funções para nos auxiliar, como conversores

In [ ]:
def bbox_to_tensor(bbox):
    """
    Convert bounding box coordinates to a tensor.

    Parameters:
    bbox (tuple): Bounding box coordinates (x_center, y_center, width, height).

    Returns:
    torch.Tensor: Tensor representation of the bounding box.
    """
    x_center, y_center, width, height = bbox
    return torch.tensor([x_center, y_center, width, height], dtype=torch.float32)

def tensor_to_bbox(tensor):
    """
    Convert tensor to bounding box coordinates.

    Parameters:
    tensor (torch.Tensor): Tensor representation of the bounding box.

    Returns:
    tuple: Bounding box coordinates (x_center, y_center, width, height).
    """
    return tuple(tensor.tolist())


Temos a seguinte lógica para determinar:


```
 inSideLeftAbove(B,A) ← p(A,B),left(B,A),above(B,A)
 inAbove(C,A) ← p(A,C) above(C,A)
 inSideRightAbove(D,A) ← p(A,D), left(A,D), above(D,A)
 inSideRight(E,A) ← p(A,E), left(A,D)
 inSideRightBelow(F,A) ← p(A,F),left(A,D), above(A,F)
 inBelow(G,A) ← p(A,G),above(A,G)
 inSideLeftBelow(H,A) ← p(A,H),left(H,A),above(A,H)
 inSideLeft(I,A) ← p(A,I), left(I,A).
 edgeSideLeftAbove(J,A) ← po(A,J),left(J,A),above(J,A)
 edgeAbove(K,A) ← po(A,K),above(K,A)
 edgeSideRightAbove(L,A) ← po(A,L),left(A,L),above(L,A)
 edgeSideRight(M,A) ← po(A,M),left(A,M)
 edgeSideRightBelow(N,A) ← po(A,N),left(A,N),above(A,N)
 edgeBelow(O,A) ← po(A,O),above(A,O)
 edgeSideLeftBelow(P,A) ← po(A,P),left(P,A),above(A,P)
 edgeSideLeft(Q,A) ← po(A,Q),left(Q,A)
 outSideLeftAbove(R,A) ← d(A,R),left(A,R),above(R,A)
 outAbove(S,A) ← d(A,S),above(S,A)
 outSideRightAbove(T,A) ← d(A,T),left(A,T),above(T,A)
 outSideRight(U,A) ← d(A,U),left(A,U)
 outSideRightBelow(V,A) ← d(A,W),left(A,V),above(A,V)
 outBelow(W,A) ← d(A,W),above(A,W)
 outSideLeftBelow(X,A) ← d(A,X),left(X,A),above(A,X)
 outSideLeft(Y,A) ← d(A,Y),left(Y,A)
 in(A, Z) ← p(Z, A)
```



Essa lógica, pode ser escrita em um predicado

O artigo sugere a seguinte abordagem para LTN:


*   Domain -labels denoting the relations
*   Constant are the bounding boxes
*   Predicates spatial relations of bounding boxes pairs
*   Axioms represent the mutual exclusion of labels on spatial relations
*   Grounding G (itens) = N △ vectors are used to represent class labels
*   Learning As before, the fuzzy logic operators and connectives are approximated using the
stable product configuration
*   Querying LTN constraint learning






### Definindo as constantes:

In [ ]:
A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W, X, Y, Z = [
    (tf.constant((clockwise_bounding_boxes.get(letter))))
    for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
]

In [ ]:
teste2 = bbox_to_tensor(clockwise_bounding_boxes.get('A'));

In [ ]:
A

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.5 , 0.5 , 0.02, 0.06], dtype=float32)>

###Definindo os predicados

In [ ]:
def p (A,B):

  return ltn.Predicate(O(A, B))

def po(A, B):
    return ltn.Predicate(PO(A, B))

In [ ]:
bb3 = tf.constant( clockwise_bounding_boxes.get('A') )
bb2 = tf.constant( clockwise_bounding_boxes.get('B') )

In [ ]:
#Exemplo de teste
O(bb3,bb2)

#print(result)

<tf.Tensor: shape=(), dtype=bool, numpy=False>

In [ ]:
bounding_boxes_tensors = {key: torch.tensor(value, dtype=torch.float32) for key, value in clockwise_bounding_boxes.items()}
bounding_boxes_tensors


In [ ]:
A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W, X, Y, Z = [
    ltn.Variable(name, tensor) for name,
    tensor in bounding_boxes_tensors.items()]
